<a id='sect0'></a>
## <b><font color='darkblue'>Preface</font></b>
([article source](https://towardsdatascience.com/intro-to-llm-agents-with-langchain-when-rag-is-not-enough-7d8c08145834)) <b><font size='3ptx'>This article is a written form of a tutorial I conducted two weeks ago with Neurons Lab. If you prefer a narrative walkthrough, you can find the [YouTube video here](https://www.youtube.com/watch?v=uVkS05qPhik)</font></b>

As always, you can find [the code on GitHub](https://github.com/Rachnog/intro_to_llm_agents), and here are separate Colab Notebooks:
* <b>[Planning and reasoning](#sect2)</b> ([colab](https://colab.research.google.com/drive/1SplDwEIbVfo9zNt6JOK0gJlV0wCNuz0F?usp=sharing))
* <b>[Different types of memories](#sect3)</b> ([colab](https://colab.research.google.com/drive/13b_pD27aqcNXYI2M7fBxK1fRIO2pygZJ?usp=sharing))
* <b>[Various types of tools](#sect4)</b> ([colab](https://colab.research.google.com/drive/1-VpwkmSvzA-zQ_iVK-xjOQf5kA-Lzwg9?usp=sharing))
* <b>[Building complete agents](#sect5)</b> ([colab](https://colab.research.google.com/drive/1aC9AUNNYYz36atE8BUJH4fZIfknHa9Pk?usp=sharing))

### <b><font color='darkgreen'>Agenda</font></b>
* <b><font size='3ptx'><a href='#sect1'>Introduction to the agents</a></font></b>
* <b><font size='3ptx'><a href='#sect2'>Planning and reasoning</a></font></b>
* <b><font size='3ptx'><a href='#sect3'>Different types of memories</a></font></b>
* <b><font size='3ptx'><a href='#sect4'>Various types of tools</a></font></b>
* <b><font size='3ptx'><a href='#sect5'>Building complete agents</a></font></b>

### <b><font color='darkgreen'>Importing necessary packages</font></b>

In [140]:
from langchain import hub
from langchain.chains import SequentialChain
from langchain.chains.llm import LLMChain
from langchain_community.chat_models.openai import ChatOpenAI
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.agents import AgentExecutor
from langchain.agents import create_openai_functions_agent
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.tools.tavily_search import TavilySearchResults
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

from dotenv import load_dotenv

load_dotenv()

True

<a id='sect1'></a>
## <b><font color='darkblue'>Introduction to the agents</font></b>
<b><font size='3ptx'>Let’s begin the lecture by exploring various examples of LLM agents.</font></b>
* <font size='3ptx'><b><a href='#sect1_1'>Step 1: Planning</a></b></font>
* <font size='3ptx'><b><a href='#sect1_2'>Step 2: Memory</a></b></font>
* <font size='3ptx'><b><a href='#sect1_3'>Step 3: Tools</a></b></font>
* <font size='3ptx'><b><a href='#sect1_4'>Step 4: All together</a></b></font>

![fig1](images/fig1.PNG)

While the topic is widely discussed, few are actively utilizing agents; often, what we perceive as agents are simply large language models. Let’s consider such a simple task as searching for football game results and saving them as a CSV file. We can compare several available tools:
* **GPT-4 with search and plugins**: as you will find in the [chat history here](https://chat.openai.com/share/2ecd61a9-dbd9-4287-aa75-14618106a34c), GPT-4 failed to do the task due to code errors.
* **AutoGPT through** https://evo.ninja/ at least could generate some kind of CSV (not ideal though).
* **AgentGPT** through https://agentgpt.reworkd.ai/: decided to treat this task as a synthetic data generator which is not what we asked about, check the [chat history here](https://agentgpt.reworkd.ai/agent?id=clsyfh1t101t9jv08yaof890k).

**Since the available tools are not great, let’s learn from the first principles of how to build agents from scratch.** I am using amazing [Lilian’s blog article](https://lilianweng.github.io/posts/2023-06-23-agent/) as a structure reference but adding more examples on my own.

<a id='sect1_1'></a>
### <b><font color='darkgreen'>Step 1: Planning</font></b>
<b><font size='3ptx'>You might have come across various techniques aimed at improving the performance of large language models, such as [offering tips](https://twitter.com/literallydenis/status/1730965217125839142) or even jokingly threatening them.</font></b>

![fig2](images/fig2.PNG)

**One popular technique is called “[chain of thought,](https://www.promptingguide.ai/techniques/cot)” where the model is asked to think step by step, enabling self-correction**. This approach has evolved into more advanced versions like the “[**chain of thought with self-consistency**](https://www.promptingguide.ai/techniques/consistency)” and the generalized “[**tree of thoughts,**](https://medium.com/@astropomeai/implementing-the-tree-of-thoughts-in-langchains-chain-f2ebc5864fac)” (ToT) <b>where multiple thoughts are created, re-evaluated, and consolidated to provide an output</b>.

**In this tutorial, I am using heavily [Langsmith](https://www.langchain.com/langsmith), a platform for productionizing LLM applications**. For example, while building the tree of thoughts prompts, I save my sub-prompts in the prompts repository and load them:

In [8]:
cot_step1 = hub.pull("rachnogstyle/nlw_jan24_cot_step1")
cot_step2 = hub.pull("rachnogstyle/nlw_jan24_cot_step2")
cot_step3 = hub.pull("rachnogstyle/nlw_jan24_cot_step3")
cot_step4 = hub.pull("rachnogstyle/nlw_jan24_cot_step4")

model = "gpt-3.5-turbo"

# Please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter
chain1 = LLMChain(
    llm=ChatOpenAI(temperature=0, model=model),
    prompt=cot_step1,
    output_key="solutions"
)

chain2 = LLMChain(
    llm=ChatOpenAI(temperature=0, model=model),
    prompt=cot_step2,
    output_key="review"
)

chain3 = LLMChain(
    llm=ChatOpenAI(temperature=0, model=model),
    prompt=cot_step3,
    output_key="deepen_thought_process"
)

chain4 = LLMChain(
    llm=ChatOpenAI(temperature=0, model=model),
    prompt=cot_step4,
    output_key="ranked_solutions"
)

overall_chain = SequentialChain(
    chains=[chain1, chain2, chain3, chain4],
    input_variables=["input", "perfect_factors"],
    output_variables=["ranked_solutions"],
    verbose=True
)

/usr/local/google/home/johnkclee/Github/ml_articles/env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


You can see in [this notebook](https://github.com/Rachnog/intro_to_llm_agents/blob/main/1_planning.ipynb) the result of such reasoning, <b>the point I want to make here is the right process for defining your reasoning steps and versioning them in such an LLMOps system like [Langsmith](https://www.langchain.com/langsmith)</b>.

Also, you can see other examples of popular reasoning techniques in public repositories like ReAct or Self-ask with search:

```python
prompt = hub.pull("hwchase17/react")
prompt = hub.pull("hwchase17/self-ask-with-search")
```

Other notable approaches are:
* **Reflexion** ([Shinn & Labash 2023](https://arxiv.org/abs/2303.11366)) is a framework to equip agents with dynamic memory and self-reflection capabilities to improve reasoning skills.
* **Chain of Hindsight** ([CoH; Liu et al. 2023](https://arxiv.org/abs/2302.02676)) encourages the model to improve on its own outputs by explicitly presenting it with a sequence of past outputs, each annotated with feedback.

<a id='sect1_2'></a>
### <b><font color='darkgreen'>Step 2: Memory</font></b> ([back](#sect1))
![fig3](images/fig3.PNG)

* **Sensory Memory**: This component of memory captures immediate sensory inputs, like what we see, hear or feel. In the context of prompt engineering and AI models, a prompt serves as a transient input, similar to a momentary touch or sensation. It’s the initial stimulus that triggers the model’s processing.
* **Short-Term Memory**: Short-term memory holds information temporarily, typically related to the ongoing task or conversation. In prompt engineering, this equates to retaining the recent chat history. This memory enables the agent to maintain context and coherence throughout the interaction, ensuring that responses align with the current dialogue. In code, you typically add it as conversation history:
```python
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.agents import AgentExecutor
from langchain.agents import create_openai_functions_agent

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
tools = [retriever_tool]
agent = create_openai_functions_agent(
    llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

message_history = ChatMessageHistory()
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)
```

* **Long-Term Memory**: Long-term memory stores both factual knowledge and procedural instructions. In AI models, this is represented by the data used for training and fine-tuning. Additionally, long-term memory supports the operation of RAG frameworks, allowing agents to access and integrate learned information into their responses. It’s like the comprehensive knowledge repository that agents draw upon to generate informed and relevant outputs. In code, you typically add it as a vectorized database:
```python
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

loader = WebBaseLoader("https://neurons-lab.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()
```

<a id='sect1_3'></a>
### <b><font color='darkgreen'>Step 3: Tools</font></b> ([back](#sect1))
<b><font size='3ptx'>ChatGPT [Plugins](https://openai.com/blog/chatgpt-plugins) and OpenAI API [function calling](https://platform.openai.com/docs/guides/gpt/function-calling) are good examples of LLMs augmented with tool use capability working in practice.

![fig4](images/fig4.PNG)

* **Built-in Langchain tools**: Langchain has a [plenty of built-in tools](https://python.langchain.com/docs/integrations/tools/) ranging from internet search and Arxiv toolkit to Zapier and Yahoo Finance. For this simple tutorial, we will experiment with the internet search provided by [**Tavily**](https://tavily.com/):
```python
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.tools.tavily_search import TavilySearchResults

search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)
agent_chain = initialize_agent(
    [retriever_tool, tavily_tool],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)
```

* **Custom tools**: it’s also very easy to define your own tools. Let’s dissect the simple example of a tool that calculates the length of the string. You need to use the <b><font color='violet'>@tool</font> decorator to make Langchain know about it. Then, don’t forget about the type of input and the output. But the most important part will be the function comment between """ """ — this is how your agent will know what this tool does and will compare this description to descriptions of the other tools:
```python
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def calculate_length_tool(a: str) -> int:
    """The function calculates the length of the input string."""
    return len(a)

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)
agent_chain = initialize_agent(
    [retriever_tool, tavily_tool, calculate_length_tool],
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)
```

You can find examples of how it works [in this script](https://github.com/Rachnog/intro_to_llm_agents/blob/main/3_tools.ipynb), **but you also can see an error** — it doesn't pull the correct description of the Neurons Lab company and despite calling the right custom function of length calculation, the final result is wrong. Let’s try to fix it!

<a id='sect1_4'></a>
### <b><font color='darkgreen'>Step 4: All together</font></b> ([back](#sect1))
<b><font size='3ptx'>I am providing a clean version of combining all the pieces of architecture together in this script.</font></b>

Notice, how we can easily decompose and define separately:
* All kinds of **tools** (<font color='brown'>search, custom tools, etc</font>)
* All kinds of **memories** (<font color='brown'>**sensory** as a prompt, **short-term** as runnable message history, and as a sketchpad within the prompt, and **long-term** as a retrieval from the vector database</font>)
* Any kind of **planning strategy** (<font color='brown'>as a part of a prompt pulled from the LLMOps system</font>)

The final definition of the agent will look as simple as this:
```python
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)
```

As you can see in the [outputs of the script](https://github.com/Rachnog/intro_to_llm_agents/blob/main/4_agents.ipynb) (<font color='brown'>or you can run it yourself</font>), it solves the issue in the previous part related to tools. What changed? We defined a **complete architecture**, where short-term memory plays a crucial role. Our agent obtained **message history and a sketchpad as a part of the reasoning structure** which allowed it to pull the correct website description and calculate its length.

<a id='sect2'></a>
## <b><font color='darkblue'>Planning and reasoning</font></b> ([back](#sect0))
([source colab](https://colab.research.google.com/drive/1SplDwEIbVfo9zNt6JOK0gJlV0wCNuz0F?usp=sharing)) <b><font size='3ptx'>Here we are going to run through several common planning and reasoning framework to see how they work.</font></b>

In [14]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [15]:
from langchain import hub

cot_step1 = hub.pull("rachnogstyle/nlw_jan24_cot_step1")
cot_step2 = hub.pull("rachnogstyle/nlw_jan24_cot_step2")
cot_step3 = hub.pull("rachnogstyle/nlw_jan24_cot_step3")
cot_step4 = hub.pull("rachnogstyle/nlw_jan24_cot_step4")

In [16]:
model = "gpt-3.5-turbo"

chain1 = LLMChain(
    llm=ChatOpenAI(temperature=0, model=model),
    prompt=cot_step1,
    output_key="solutions"
)

chain2 = LLMChain(
    llm=ChatOpenAI(temperature=0, model=model),
    prompt=cot_step2,
    output_key="review"
)

chain3 = LLMChain(
    llm=ChatOpenAI(temperature=0, model=model),
    prompt=cot_step3,
    output_key="deepen_thought_process"
)

chain4 = LLMChain(
    llm=ChatOpenAI(temperature=0, model=model),
    prompt=cot_step4,
    output_key="ranked_solutions"
)

In [24]:
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[chain1, chain2, chain3, chain4],
    input_variables=["input", "perfect_factors"],
    output_variables=["ranked_solutions"],
    verbose=True
)

output = overall_chain(
    {
        "input": "How to become a successful software engineer?",
        "perfect_factors": "It is difficult for a company to hire an engineer without experience."
    })



> Entering new SequentialChain chain...

> Finished chain.


In [25]:
output.keys()

dict_keys(['input', 'perfect_factors', 'ranked_solutions'])

In [26]:
print(output['ranked_solutions'])

Ranking: 

1. Participating in a semester-long co-op program with a software development firm
2. Securing a summer internship at a tech company
3. Taking on a part-time internship during the school year

Justification:

1. Participating in a semester-long co-op program with a software development firm ranks highest because it offers a longer and more immersive experience in a professional setting, allowing for deeper learning and skill development. It also provides the opportunity to build strong relationships with industry professionals and potentially secure a job offer after graduation.

2. Securing a summer internship at a tech company is ranked second as it still provides valuable hands-on experience in a shorter time frame. It allows students to gain exposure to the industry and build their resume, potentially leading to future internship or job opportunities.

3. Taking on a part-time internship during the school year is ranked last as it may be more challenging to balance inter

### <b><font color='darkgreen'>ReAct prompt overview</font></b>
Let's review [**ReAct**](https://www.google.com/url?q=https%3A%2F%2Fpython.langchain.com%2Fdocs%2Fmodules%2Fagents%2Fagent_types%2Freact) prompt as it's defined in Langchain.

In [27]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")

'Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

In [28]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


### <b><font color='darkgreen'>Self-ask with search</font></b>
Let's review [self-ask with search](https://www.google.com/url?q=https%3A%2F%2Fpython.langchain.com%2Fdocs%2Fmodules%2Fagents%2Fagent_types%2Fself_ask_with_search) as it's defined in Langchain.

In [29]:
prompt = hub.pull("hwchase17/self-ask-with-search")

In [30]:
print(prompt.template)

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin

### <b><font color='darkgreen'>Approaches to read next</font></b>
* **Reflexion** ([Shinn & Labash 2023](https://www.google.com/url?q=https%3A%2F%2Farxiv.org%2Fabs%2F2303.11366)) is a framework to equips agents with dynamic memory and self-reflection capabilities to improve reasoning skills.
* **Chain of Hindsight** ([CoH; Liu et al. 2023](https://www.google.com/url?q=https%3A%2F%2Farxiv.org%2Fabs%2F2302.02676)) encourages the model to improve on its own outputs by explicitly presenting it with a sequence of past outputs, each annotated with feedback.

<a id='sect3'></a>
## <b><font color='darkblue'>Different types of memories</font></b> ([back](#sect0))
![fig3](images/fig3.PNG)

- **Sensory Memory:** This component of memory captures immediate sensory inputs, like what we see, hear, or feel. In the context of prompt engineering and AI models, a prompt serves as a transient input, similar to a momentary touch or sensation. It's the initial stimulus that triggers the model's processing.

- **Short-Term Memory:** Short-term memory holds information temporarily, typically related to the ongoing task or conversation. In prompt engineering, this equates to retaining the recent chat history. This memory enables the agent to maintain context and coherence throughout the interaction, ensuring that responses align with the current dialogue.

- **Long-Term Memory:** Long-term memory stores both factual knowledge and procedural instructions. In AI models, this is represented by the data used for training and fine-tuning. Additionally, long-term memory supports the operation of RAG frameworks, allowing agents to access and integrate learned information into their responses. It's like the comprehensive knowledge repository that agents draw upon to generate informed and relevant outputs.

### <b><font color='darkgreen'>Adding long-term memory</font></b>

In [32]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

In [31]:
loader = WebBaseLoader("https://neurons-lab.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)

In [33]:
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [35]:
results = retriever.get_relevant_documents("What are the projects in healthcare?")

In [39]:
results[0].page_content

'Creative Practice Solutions : Developing an AI-Driven Medical Transcription & Billing System                                                \n\n\nHealthTech\n\n\n\nExplore story\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMagellan X: Creating a predictive maintenance solution in shipping with Magellan X                                                \n\n\nCleantech\n\n\n\nExplore story\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\niPlena: Transforming user experience in physiotherapy                                                \n\n\nHealthTech\n\n\n\nExplore story\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHealthTech client: Building a Remote Patient Monitoring system for health providers                                                \n\n\nHealthTech\n\n\n\nExplore story\n\n\n\n\n\n\n\n\n\n\n\n\nCleanTech client: Developing Machine learning models to predict PV output and consumption                                                \n\n\nCleantech\n\n\n\nExplore story'

### <b><font color='darkgreen'>Adding short-term memory</font></b>

In [43]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.agents import AgentExecutor
from langchain.agents import create_openai_functions_agent
from langchain.tools.retriever import create_retriever_tool
from langchain_openai import ChatOpenAI
from langchain import hub

In [44]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

In [45]:
print(prompt.messages)

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(variable_name='agent_scratchpad')]


In [46]:
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [47]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
tools = [retriever_tool]
agent = create_openai_functions_agent(
    llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

message_history = ChatMessageHistory()
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [48]:
agent_with_chat_history.invoke(
    {"input": "Hi! I'm John Lee"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)

Parent run 2bcd31df-b73c-4aa1-9cf2-30a9807e0bb5 not found for run e5edba71-d536-4229-af62-cb114d1e7eeb. Treating as a root run.




> Entering new AgentExecutor chain...
Hello John Lee! How can I assist you today?

> Finished chain.


{'input': "Hi! I'm John Lee",
 'chat_history': [],
 'output': 'Hello John Lee! How can I assist you today?'}

In [49]:
output = agent_with_chat_history.invoke(
    {"input": "Do you know what's my name?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)

Parent run e37d290f-879f-4537-b397-a33a40328939 not found for run 521670d7-f86f-4d83-ac7b-875f1396fbbe. Treating as a root run.




> Entering new AgentExecutor chain...
Yes, you mentioned that your name is John Lee. How can I help you, John?

> Finished chain.


In [51]:
output.keys()

dict_keys(['input', 'chat_history', 'output'])

In [52]:
output['chat_history']

[HumanMessage(content="Hi! I'm John Lee"),
 AIMessage(content='Hello John Lee! How can I assist you today?')]

In [53]:
print(output['output'])

Yes, you mentioned that your name is John Lee. How can I help you, John?


<a id='sect4'></a>
## <b><font color='darkblue'>Various types of tools</font></b> ([back](#sect0))
<b><font size='3ptx'>First such tool-augmented architecture started in 2022 with A21Labs</font></b>
* <b><a href='#sect4_1'>Memory as a retrieval tool</a></b>
* <b><a href='#sect4_2'>Langchain tools</a></b>
* <b><a href='#sect4_3'>Custom tools and reasoning</a></b>

![fig4](images/fig4.PNG)

**These modules can be neural** (e.g. deep learning models) **or symbolic** (e.g. math calculator, currency converter, weather API). ChatGPT [Plugins](https://www.google.com/url?q=https%3A%2F%2Fopenai.com%2Fblog%2Fchatgpt-plugins) and OpenAI [API function](https://www.google.com/url?q=https%3A%2F%2Fplatform.openai.com%2Fdocs%2Fguides%2Fgpt%2Ffunction-calling) calling are good examples of LLMs augmented with tool use capability working in practice

<a id='sect4_1'></a>
### <b><font color='darkgreen'>Memory as a retrieval tool</font></b>
* [langchain.tools.retriever.create_retriever_tool](https://api.python.langchain.com/en/latest/tools/langchain.tools.retriever.create_retriever_tool.html)

In [54]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool

In [55]:
loader = WebBaseLoader("https://neurons-lab.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)

In [56]:
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [57]:
tools_description = "Search for information about Neurons Lab. For any questions about Neurons Lab, you must use this tool!"
retriever_tool = create_retriever_tool(
    retriever,  # The retriever to use for the retrieval
    "neurons_lab_search",  # The name for the tool. This will be passed to the language model, so should be unique and somewhat descriptive.
    tools_description,  # The description for the tool. This will be passed to the language model, so should be descriptive.
)

<a id='sect4_2'></a>
### <b><font color='darkgreen'>Langchain tools</font></b> ([back](#sect4))
More tools here https://python.langchain.com/docs/integrations/tools

In [58]:
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.tools.tavily_search import TavilySearchResults
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI

In [61]:
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)

In [62]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)
agent_chain = initialize_agent(
    [retriever_tool, tavily_tool],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

/usr/local/google/home/johnkclee/Github/ml_articles/env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [63]:
# run the agent
output = agent_chain.run(
    "What are the Neurons Lab services",
)

/usr/local/google/home/johnkclee/Github/ml_articles/env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: I should use the Neurons Lab search tool to find information about the services they offer.

Action:
```
{
  "action": "neurons_lab_search",
  "action_input": "services offered"
}
```
Observation: Grow your business & attract clients 
Do you also seek development opportunities or funding? Thanks to our advanced-tier AWS partnership and access to a vast VC network, we can aid your company in obtaining grants and other assistance.

















Talk to our expert team from Europe
Get in touch








Talk to our expert team from USA
Get in touch





Articles = mass of insights /
volume of experience

ALL POSTINGS

                                    1 — 6                                    











AI for portfolio management: from Markowitz to Reinforcement Learning

                                                    Jul 16, 2023|11 min read                                                


FinTech









Accelerate Remote Patie

In [66]:
print(output)

The Neurons Lab offers services such as AI feasibility analysis, AI solution engineering, AI operations management, Generative AI Solutions, and GenAI Workshop + PoC. They also collaborate with companies in various industries and can assist in seeking development opportunities or funding through their partnerships and network.


<a id='sect4_3'></a>
### <b><font color='darkgreen'>Custom tools and reasoning</font></b> ([back](#sect4))
* Here is a full guide: https://python.langchain.com/docs/modules/agents/tools/custom_tools
* Reasoning: https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_types.AgentType.html

In [67]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

In [86]:
@tool
def calculate_magic_num(a: int) -> float:
    """The function calculates the magic number of input integer `a`."""
    return int(a) * 2 + 1

In [87]:
calculate_magic_num.description, calculate_magic_num.args,

('calculate_magic_num(a: int) -> float - The function calculates the magic number of input integer `a`.',
 {'a': {'title': 'A', 'type': 'integer'}})

In [88]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)
agent_chain = initialize_agent(
    [retriever_tool, tavily_tool, calculate_magic_num],
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [89]:
# run the agent
output = agent_chain.run(
    "Give me the magic number calculated by number 12.",
)



> Entering new AgentExecutor chain...
I need to use the calculate_magic_num function to find the magic number for the input number 12.
Action: calculate_magic_num
Action Input: 12
Observation: 25
Thought:I now know the final answer
Final Answer: 25

> Finished chain.


In [90]:
print(output)

25


<a id='sect5'></a>
## <b><font color='darkblue'>Building complete agents</font></b> ([back](#sect0))
<b><font size='3ptx'>Now it is time to put all things together as an agent.</font></b>
* <font size='3ptx'><b>[Tools](#sect5_1)</b></font>
* <font size='3ptx'><b>[Memory](#sect5_2)</b></font>
* <font size='3ptx'><b>[Agent](#sect5_3)</b></font>

<a id='sect5_1'></a>
### <b><font color='darkgreen'>Tools</font></b> ([back](#sect5))
Here we define some tools to support agent for our customized operations.

In [91]:
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.tools.tavily_search import TavilySearchResults
from langchain.tools import tool

In [92]:
# Built-in searching tool
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)

In [93]:
# Customized tools
@tool
def calculate_magic_number(a: str) -> int:
    """The function calculates the magic number of input string as integer."""
    return int(a) * 2 + 3

@tool
def calculate_uppercase_tool(a: str) -> int:
    """The function calculates the number of uppercase characters in the input string."""
    return sum(1 for c in a if c.isupper())

<a id='sect5_2'></a>
### <b><font color='darkgreen'>Memory</font></b> ([back](#sect5))
Let's enable our agent to be capable of both short-term and long-term meory.

In [94]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool

#### <b>Long-term memory</b>

In [95]:
loader = WebBaseLoader("https://neurons-lab.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())

In [96]:
retriever = vector.as_retriever()
retriever_tool = create_retriever_tool(
    retriever,
    "neurons_lab_search",
    "Search for information about Neurons Lab. For any questions about Neurons Lab, you must use this tool!",
)

#### <b>Short-term memory</b>

In [97]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [98]:
message_history = ChatMessageHistory()

#### <b>Sensory memory</b>
https://smith.langchain.com/hub/hwchase17/openai-functions-agent

In the context of humans, sensory memory is a very brief, almost instantaneous, buffer for sensory information. It acts like a fleeting snapshot of the world experienced through our senses (sight, sound, touch, etc.). It holds this information just long enough for the brain to decide what to pay attention to and process further.

When it comes to large language models (LLMs) like me, the concept of sensory memory isn't directly applicable in the same way it is to humans. This is because we lack physical senses and don't experience the world through sight, sound, or touch.

In [99]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain import hub

In [100]:
prompt = hub.pull("hwchase17/openai-functions-agent")

In [101]:
print(prompt)

input_variables=['agent_scratchpad', 'input'] input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(variable_name='agent_

<a id='sect5_3'></a>
### <b><font color='darkgreen'>Agent</font></b> ([back](#sect5))
It's time to glue tools, memory along with an angent for our usage.

#### <b>All tools together</b>

In [121]:
tools = [retriever_tool, tavily_tool, calculate_magic_number, calculate_uppercase_tool]
for t in tools:
    print(f'- Tool ({t.name}): {t.description}')

- Tool (neurons_lab_search): Search for information about Neurons Lab. For any questions about Neurons Lab, you must use this tool!
- Tool (tavily_search_results_json): A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.
- Tool (calculate_magic_number): calculate_magic_number(a: str) -> int - The function calculates the magic number of input string as integer.
- Tool (calculate_uppercase_tool): calculate_uppercase_tool(a: str) -> int - The function calculates the number of uppercase characters in the input string.


#### <b>Defining an agent with tools and memory</b>
* [**API**:langchain.agents.openai_functions_agent.base.create_openai_functions_agent](https://api.python.langchain.com/en/latest/agents/langchain.agents.openai_functions_agent.base.create_openai_functions_agent.html)
* [**class**:langchain.agents.agent.AgentExecutor](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent.AgentExecutor.html)
* [**class**:langchain_core.runnables.history.RunnableWithMessageHistory](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html)

In [122]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

In [123]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

#### <b>Running an agent</b>

In [124]:
output = agent_with_chat_history.invoke(
    {
        "input": "Hello, my name is John Lee!"
    },
    config={"configurable": {"session_id": "<foo>"}},
)

Parent run fe2f1d7d-e306-4f03-8134-85ab5392ea05 not found for run 37440ff8-b52b-48de-b096-2aa0e2a80b1b. Treating as a root run.




> Entering new AgentExecutor chain...
Hello John Lee! It's nice to meet you. How can I help you today?

> Finished chain.


In [108]:
output.keys()

dict_keys(['input', 'chat_history', 'output'])

In [125]:
print(output['output'])

Hello John Lee! It's nice to meet you. How can I help you today?


In [113]:
output = agent_with_chat_history.invoke(
    {
        "input": "Please find the upper case characters in my name."
    },
    config={"configurable": {"session_id": "<foo>"}},
)

Parent run 8910e34f-67f8-4988-969e-5cb16689c5b3 not found for run 39ad7d94-fe1b-4e86-9b3f-21c9e48a90fb. Treating as a root run.




> Entering new AgentExecutor chain...

Invoking: `calculate_uppercase_tool` with `{'a': 'John Lee'}`


2There are 2 uppercase characters in your name "John Lee".

> Finished chain.


In [114]:
print(output['output'])

There are 2 uppercase characters in your name "John Lee".


In [126]:
output = agent_with_chat_history.invoke(
    {
        "input": "Use the number of upper case character of my name to calculate the magic number."
    },
    config={"configurable": {"session_id": "<foo>"}},
)

Parent run 3de8e223-80b3-4e44-bec6-281013d0cc93 not found for run 0e763e97-ab5c-4f53-b109-3ea0780f46aa. Treating as a root run.




> Entering new AgentExecutor chain...

Invoking: `calculate_uppercase_tool` with `{'a': 'John Lee'}`


2
Invoking: `calculate_magic_number` with `{'a': '2'}`


7The magic number calculated using the number of uppercase characters in your name "John Lee" is 7.

> Finished chain.


In [127]:
print(output['output'])

The magic number calculated using the number of uppercase characters in your name "John Lee" is 7.


#### <b>Structuring outputs</b>

In [128]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field, validator

In [129]:
# Define your desired data structure.
class DesiredStructure(BaseModel):
    question: str = Field(description="the question asked")
    numerical_answer: int = Field(description="the number extracted from the answer, text excluded")
    text_answer: str = Field(description="the text part of the answer, numbers excluded")
parser = PydanticOutputParser(pydantic_object=DesiredStructure)

In [137]:
model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
prompt_and_model = prompt | model

In [138]:
output = prompt_and_model.invoke({
    "query": "Calculate the number of character in word 'Google' and double the number."}
)

In [139]:
print(output.content)

{
  "question": "Calculate the number of character in word 'Google' and double the number.",
  "numerical_answer": 12,
  "text_answer": "The word 'Google' has 6 characters. Doubling it gives 12."
}


#### <b>others</b>
Next stes: evaluation and benchmarks https://python.langchain.com/docs/langsmith/walkthrough

## <b><font color='darkblue'>Supplement</font></b>
* [Medium - Implementing the Tree of Thoughts in LangChain’s Chain](https://medium.com/@astropomeai/implementing-the-tree-of-thoughts-in-langchains-chain-f2ebc5864fac)
* [The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use **RunnableSequence**](https://github.com/Azure-Samples/chat-with-your-data-solution-accelerator/issues/998)